In [ ]:
# After testing, I now understand that pyspark dataframes
#  do not support indexing, so access will be faster under some 
#  circumstances with a python spark connector
#
#
#

In [2]:
# trying this again after adding index for year

# play with some sql selections to make sure things are working

from pytorch_pretrained_bert import BertModel, BertTokenizer
import torch

import time
import mysql.connector as mysql

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import Row, StructType, StructField, IntegerType, StringType

import os
import time
import random
#import mysql.connector as mysql   # import gc




import csv

#import igraph

import time
import numpy as np

import matplotlib.pyplot as plt
#import cairocffi as cairo

import seaborn as sns
import umap
import statsmodels.api as sm  # for kdemultivariate 




Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [12]:
SUBMIT_ARGS = "--driver-class-path file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar --jars file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

db_name = 'test_pubmed'  # db name collisons? https://stackoverflow.com/questions/14011968/user-cant-access-a-database
url = "jdbc:mysql://localhost:3306/{}?useUnicode=true&useJDBCCompliantTimezoneShift=true&useLegacyDatetimeCode=false&serverTimezone=America/Chicago".format(db_name)  # mysql runs on port 3306
client_config = {'unix_socket':'/home/brendanchambers/.sql.sock',
                'database': db_name}  # for python connector

In [4]:
print('initializing spark')
# init spark
conf = SparkConf()
conf = (conf.setMaster('local[*]')
       .set('spark.driver.memory','28G')
       .set("spark.jars", "/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar"))        
'''
.set('spark.executor.memory','1G')  # 20
.set('spark.driver.memory','1G')   # 40
.set('spark.driver.maxResultSize','500M')  #.set('spark.storage.memoryFraction',0))  # this setting is now a legacy option
.set('spark.python.worker.reuse', 'false')
.set('spark.python.worker.memory','512m')
.set('spark.executor.cores','1'))
'''
sc = SparkContext(conf=conf)
#sc.addJar('home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar')  # temp
spark = SparkSession(sc)  # don't need this for vanilla RDDs

print(sc._conf.getAll())

initializing spark
[('spark.repl.local.jars', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.app.id', 'local-1565021447940'), ('spark.jars', '/home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.executor.id', 'driver'), ('spark.app.name', 'pyspark-shell'), ('spark.rdd.compress', 'True'), ('spark.driver.extraClassPath', 'file:///home/brendanchambers/my_resources/mysql-connector-java-8.0.16/mysql-connector-java-8.0.16.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.host', 'midway2-0385.rcc.local'), ('spark.master', 'local[*]'), ('spark.driver.memory', '28G'), ('spark.driver.port', '45943'), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [8]:
# test read

tablename_ = 'metadata'

sql = "(SELECT * FROM {}) AS t".format(tablename_)
print(sql)

start_time = time.time()
df_meta = spark.read.format('jdbc').option("url", url)\
                              .option("dbtable", sql)\
                              .load().repartition(9).cache()
end_time = time.time()
print("dataframe loaded in {} s".format(end_time - start_time))

print(df_meta.rdd.getNumPartitions())

(SELECT * FROM metadata) AS t
dataframe loaded in 0.04593849182128906 s
9


In [20]:
# for a single year

START_YEAR = 2008  # inclusive
END_YEAR = 2009  # exclusive
year_set = []
for year in range(START_YEAR, END_YEAR):
    year_set.append(str(year))
print(year_set)

start_time = time.time()

papers_for_year = df_meta.filter(df_meta.year.isin(year_set)).cache()

end_time = time.time()
print("elapsed: {}".format(end_time - start_time))


start_time = time.time()
N_papers = papers_for_year.count()
print("N papers: {}".format(N_papers))
end_time = time.time()
print("elapsed: {}".format(end_time - start_time))

start_time = time.time()
output = papers_for_year.collect()
for idx, row in enumerate(output):
    print(row)
    print()
    if idx > 15:
        break
end_time = time.time()
print("elapsed: {}".format(end_time - start_time))

['2008']
elapsed: 0.08839583396911621
N papers: 750874
elapsed: 4.2200822830200195
Row(pmid=18504544, year=2008, human='1', animal='0', molcel='0', xcoord='0', ycoord='1', approx_potential_translate='0.5', cited_by_clin='0', title='Autoantibodies to islet antigen-2 are associated with HLA-DRB1*07 and DRB1*09 haplotypes as well as DRB1*04 at onset of type 1 diabetes: the possible role of HLA-DQA in autoimmunity to IA-2.', authors='A J K Williams, R J Aitken, M A-M Chandler, K M Gillespie, V Lampasona, P J Bingley', journal='Diabetologia', article='1', is_clinical='0', estimated='0', relative_citation_ratio='0.715492507534')

Row(pmid=19014095, year=2008, human='0', animal='1', molcel='0', xcoord='0.866', ycoord='-0.5', approx_potential_translate='0.05', cited_by_clin='0', title='Bronchogenic adenocarcinoma in a hyacinth macaw (Anodorhynchus hyacinthinus).', authors='Wes A Baumgartner, David Sanchez-Migallon Guzman, Shawn Hollibush, Lorrie Gaschen, E Clay Hodgin, Mark A Mitchell', journa

In [21]:
# try the same query with python mysql connector

# RESULTS this is slower than the pyspark dataframe method
#   suggesting that the indexed columns are being used smartly by pyspark
#   while also utilizing parallelization


year_ = 2008
sql = 'SELECT pmid, title, year FROM metadata WHERE year = {};'.format(year_)

db = mysql.connect(**client_config)
cursor = db.cursor()
start_time = time.time()
cursor.execute(sql)
print('elapsed time: {}'.format(time.time() - start_time))

start_time = time.time()
output = cursor.fetchall()
for idx,row in enumerate(output):
    print(row)
    print()
    if idx > 15:
        break
print('elapsed time: {}'.format(time.time() - start_time))


cursor.close()

elapsed time: 0.3947865962982178
(17286970, 'Anchors aweigh: a demonstration of cross-modality anchoring and magnitude priming.', 2008)

(17287065, 'Customized birthweight standards for a Spanish population.', 2008)

(17287068, 'Assessment of spatial-temporal patterns of surface and ground water qualities and factors influencing management strategy of groundwater system in an urban river corridor of Nepal.', 2008)

(17287069, 'Environmental regulatory failure and metal contamination at the Giap Lai pyrite mine, Northern Vietnam.', 2008)

(17287070, 'Environmental restoration of invaded ecosystems: how much versus how often?', 2008)

(17287071, 'Removal of o-nitrophenol from water by electrochemical degradation using a lead oxide/titanium modified electrode.', 2008)

(17287133, 'Fatigue-induced changes in muscle fiber action potentials estimated by wavelet analysis.', 2008)

(17287140, 'Task-specific focal hand dystonia with usage of a spoon.', 2008)

(17287884, 'Nano-structured support

True